# Inference, with everything together

In [1]:
import numpy as np
np.bool = np.bool_ # bad trick to fix numpy version issue :(
import os
import sys
from natsort import natsorted

sys.path = [p for p in sys.path if '/peract/' not in p]

# Set `PYOPENGL_PLATFORM=egl` for pyrender visualizations
os.environ["DISPLAY"] = ":0"
os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,3" # Depends on your computer and available GPUs

In [2]:
### Depending on your workspace, you may already have this repository installe, otherwise clone once again
if not os.path.exists(os.path.join(os.getcwd(), 'peract_colab')):
    !git clone https://github.com/yuki1003/peract_colab.git

!cd peract_colab && git pull origin master

From github.com:yuki1003/peract_colab
 * branch              master     -> FETCH_HEAD
Already up to date.


In [3]:
# Check if modules in peract_colab repository are recognized.
try: # test import
    from rlbench.utils import get_stored_demo
except ImportError as error_message:
    print(error_message)
    print("Adding peract_colab repository to system path.")
    sys.path.append('peract_colab')

In [4]:
import torch

from agent.perceiver_io import PerceiverIO
from agent.peract_agent import PerceiverActorAgent

In [5]:
#___DATA___
TASK = 'handing_over_banana'

# Data Constants
WORKSPACE_DIR = os.getcwd()
DATA_FOLDER = os.path.join(WORKSPACE_DIR, "task_data", "handoversim")
EPISODES_FOLDER = os.path.join(TASK, "all_variations", "episodes")

EPISODE_FOLDER = 'episode%d'
SETUP = "s1" # Options: "s1"
train_data_path = os.path.join(DATA_FOLDER, f"train_{SETUP}", EPISODES_FOLDER)
TRAIN_INDEXES = [int(episode_nr.replace("episode", "")) for episode_nr in natsorted(os.listdir(train_data_path))]
test_data_path = os.path.join(DATA_FOLDER, f"val_{SETUP}", EPISODES_FOLDER)
TEST_INDEXES = [int(episode_nr.replace("episode", "")) for episode_nr in natsorted(os.listdir(test_data_path))]

print(f"TRAIN | Total #: {len(TRAIN_INDEXES)}, indices: {TRAIN_INDEXES}")
print(f"TEST | Total #: {TEST_INDEXES}")

# Replaybuffer related constants
LOW_DIM_SIZE = 4    # 4 dimensions - proprioception: {gripper_open, left_finger_joint, right_finger_joint, timestep}
IMAGE_SIZE =  128  # 128x128 - if you want to use higher voxel resolutions like 200^3, you might want to regenerate the dataset with larger images
ROTATION_RESOLUTION = 5 # degree increments per axis

DEPTH_SCALE = 1000
SCENE_BOUNDS = [0.11, -0.5, 0.8, 1.11, 0.5, 1.8]  # Must be 1m each

# Training Settings Constants
BATCH_SIZE = 1
VOXEL_SIZES = [100]  # 100x100x100 voxels
NUM_LATENTS = 512  # PerceiverIO latents: lower-dimension features of input data

CAMERAS = [f"view_{camera_i}" for camera_i in range(3)]


TRAIN | Total #: 32, indices: [45, 46, 47, 48, 49, 145, 146, 147, 148, 149, 245, 246, 247, 248, 249, 345, 346, 348, 349, 445, 446, 447, 448, 545, 546, 548, 549, 945, 946, 947, 948, 949]
TEST | Total #: [645, 646, 647, 648]


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#___AGENT___

perceiver_encoder = PerceiverIO(
    depth=6,
    iterations=1,
    voxel_size=VOXEL_SIZES[0],
    initial_dim=3 + 3 + 1 + 3,
    low_dim_size=4,
    layer=0,
    num_rotation_classes=72,
    num_grip_classes=2,
    num_collision_classes=2,
    num_latents=NUM_LATENTS,
    latent_dim=512,
    cross_heads=1,
    latent_heads=8,
    cross_dim_head=64,
    latent_dim_head=64,
    weight_tie_layers=False,
    activation='lrelu',
    input_dropout=0.1,
    attn_dropout=0.1,
    decoder_dropout=0.0,
    voxel_patch_size=5,
    voxel_patch_stride=5,
    final_dim=64,
)

peract_agent = PerceiverActorAgent(
    coordinate_bounds=SCENE_BOUNDS,
    perceiver_encoder=perceiver_encoder,
    camera_names=CAMERAS,
    batch_size=BATCH_SIZE,
    voxel_size=VOXEL_SIZES[0],
    voxel_feature_size=3,
    num_rotation_classes=72,
    rotation_resolution=ROTATION_RESOLUTION,
    image_resolution=[IMAGE_SIZE, IMAGE_SIZE],
    lambda_weight_l2=0.000001,
    transform_augmentation=False,
    rgb_augmentation=False,
    optimizer_type='lamb',
)
peract_agent.build(training=False, device=device, language_goal="handing over banana")

In [ ]:
# model_path = "/home/ywatabe/Projects/PerAct/models/2024-11-29_04-23/best_model_train" # Good
# model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-10_14-59/best_model_general" # Good {non-uniform 1 kp}
# model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-11_12-30/best_model_general" # Bad {uniform 2 kp}
# model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-11_12-30/best_model_general" # Bad
# model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-11_13-34/best_model_general"
# model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-12_17-25/best_model_general" # {crop skip 10}
# model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-12_21-23/best_model_general"
model_path = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-16_11-25/best_model_general" # {crop skip 10 new}

peract_agent.load_weights(model_path)

### Inference: Run Inference on just observation data and Save as video

In [ ]:
import imageio
import numpy as np
from matplotlib import pyplot as plt
from rlbench.utils import get_stored_demo
from rlbench.backend.utils import extract_obs
from arm.utils import visualise_voxel
from arm.utils import discrete_euler_to_quaternion, get_gripper_render_pose

# What to visualize
episode_idx_to_visualize = 646  # Index of the episode to visualize
# Video output path
video_output_path = f"demo{episode_idx_to_visualize}_visualization.mp4"

# Get demo
demo = get_stored_demo(data_path=test_data_path,
                       index=episode_idx_to_visualize,
                       cameras=CAMERAS,
                       depth_scale=DEPTH_SCALE)

episode_length = list(range(len(demo._observations)))

# Open a video writer
with imageio.get_writer(video_output_path, fps=10) as video_writer:
    for ts in episode_length[::5]:
        print(ts)
        # Extract obs at timestep
        obs_dict = extract_obs(demo._observations[ts], CAMERAS, t=ts)
        # gripper_pose = demo[ts].gripper_pose
        gripper_open = demo[ts].gripper_open
        gripper_joint_positions = demo[ts].gripper_joint_positions

        obs_dict["gripper_open"] = gripper_open
        obs_dict["gripper_joint_positions"] = gripper_joint_positions

        (continuous_trans, continuous_quat, gripper_open, _, _), \
        (voxel_grid, coord_indices, rot_and_grip_indices, gripper_open) = peract_agent.forward(obs_dict, ts)
        print(continuous_trans, continuous_quat, gripper_open)

        # Things to visualize
        vis_voxel_grid = voxel_grid[0].detach().cpu().numpy()
        pred_trans_coord = coord_indices[0].detach().cpu().numpy().tolist()

        voxel_size = 0.045
        voxel_scale = voxel_size * 100
        gripper_pose_mat = get_gripper_render_pose(voxel_scale,
                                                   SCENE_BOUNDS[:3],
                                                   continuous_trans,
                                                   continuous_quat)

        rendered_img_0 = visualise_voxel(vis_voxel_grid,
                                         None,
                                         [pred_trans_coord],
                                         None,
                                         voxel_size=voxel_size,
                                         rotation_amount=np.deg2rad(0),
                                         render_gripper=True,
                                         gripper_pose=gripper_pose_mat,
                                         gripper_mesh_scale=voxel_scale)

        rendered_img_270 = visualise_voxel(vis_voxel_grid,
                                           None,
                                           [pred_trans_coord],
                                           None,
                                           voxel_size=voxel_size,
                                           rotation_amount=np.deg2rad(45+180),
                                           render_gripper=True,
                                           gripper_pose=gripper_pose_mat,
                                           gripper_mesh_scale=voxel_scale)

        # Plot figures into a NumPy array
        fig = plt.figure(figsize=(20, 15))
        fig.add_subplot(1, 2, 1)
        plt.imshow(rendered_img_0)
        plt.axis('off')
        plt.title("Front view")
        fig.add_subplot(1, 2, 2)
        plt.imshow(rendered_img_270)
        plt.axis('off')
        plt.title("Side view")

        # Add timestamp as text with white font and black background
        fig.text(0.02, 0.95, f"Timestep: {ts}", ha='left', fontsize=16, color='white', weight='bold',
                 bbox=dict(facecolor='black', edgecolor='none', boxstyle='round,pad=0.3'))

        # Convert the matplotlib figure to a NumPy array
        fig.canvas.draw()
        img_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        
        video_writer.append_data(img_array)  # Add frame to video
        plt.close(fig)  # Close the figure to free memory

print(f"Video saved to {video_output_path}")


### Validation: Run Inference on just input data with ground truth to check what happends

In [ ]:
def get_gt_from_frame(episode_keypoints_gt_obs_dict, frame_idx):
    # Get a sorted list of dictionary keys (frames with stored data)
    episode_keypoints = sorted(episode_keypoints_gt_obs_dict.keys())

    # Iterate through the given frame indices
    # Find the smallest key that is greater than or equal to the current frame
    for episode_kp in episode_keypoints:
        if frame_idx <= episode_kp:
            episode_kp_gt_obs_dict = episode_keypoints_gt_obs_dict[episode_kp]
            return episode_kp_gt_obs_dict
    
    return None

In [ ]:
#TODO: NEED SOME WAY TO COLLECT THE GROUND TRUTH DATA FOR ALL FRAMES

import imageio
import numpy as np
from matplotlib import pyplot as plt

from rlbench.utils import get_stored_demo
from rlbench.backend.utils import extract_obs_gt

from arm.demo import _keypoint_discovery_available
from arm.utils import point_to_voxel_index, visualise_voxel, get_gripper_render_pose


# What to visualize
episode_idx_to_visualize = 645  # Index of the episode to visualize
# Video output path
video_output_path = f"demo{episode_idx_to_visualize}_gt_pred_visualization.mp4"

# Get demo
demo = get_stored_demo(data_path=test_data_path,
                       index=episode_idx_to_visualize,
                       cameras=CAMERAS,
                       depth_scale=DEPTH_SCALE)

episode_keypoints_gt_obs_dict = dict()
episode_keypoints = _keypoint_discovery_available(demo, approach_distance=0.3) #NOTE: Approach_distance Set
episode_keypoints = [episode_keypoints[-1]]
for episode_keypoint in episode_keypoints:
    episode_keypoints_gt_obs_dict[episode_keypoint] = extract_obs_gt(obs = demo._observations[episode_keypoint],
                                                                  cameras=CAMERAS)

episode_length = list(range(len(demo._observations)))
# episode_length = [40]

# Open a video writer
with imageio.get_writer(video_output_path, fps=10) as video_writer:
    for ts in episode_length[::5]: # Skip some frames
        print(ts)
        # Extract obs at timestep
        obs_dict = extract_obs(demo._observations[ts], CAMERAS, t=ts)
        # gripper_pose = demo[ts].gripper_pose
        gripper_open = demo[ts].gripper_open
        gripper_joint_positions = demo[ts].gripper_joint_positions

        obs_dict["gripper_open"] = gripper_open
        obs_dict["gripper_joint_positions"] = gripper_joint_positions

        (continuous_trans, continuous_quat, gripper_open, trans_confidence, _), \
        (voxel_grid, coord_indices, rot_and_grip_indices, gripper_open) = peract_agent.forward(obs_dict, ts)

        pred_trans_coord = coord_indices[0].detach().cpu().numpy().tolist()
        
        # Get the ground truth
        episode_keypoint_gt_obs_dict = get_gt_from_frame(episode_keypoints_gt_obs_dict, ts)
        if not (episode_keypoint_gt_obs_dict is None):
            gt_gripper_pose = episode_keypoint_gt_obs_dict["gripper_pose"]
            gt_trans_coord = point_to_voxel_index(gt_gripper_pose[:3], VOXEL_SIZES, SCENE_BOUNDS)[0]
            error = np.linalg.norm(gt_gripper_pose[:3] - continuous_trans)
            print(f"GT (voxel): {gt_trans_coord} - Prediction (voxel): {pred_trans_coord} - Error: {error} - Prediction-score: {round(trans_confidence,4)}")
        else:
            gt_trans_coord = None
            error = False
            print("GT coordinates not available for this frame")
        
        # Things to visualize
        vis_voxel_grid = voxel_grid[0].detach().cpu().numpy()

        voxel_size = 0.045
        voxel_scale = voxel_size * 100
        gripper_pose_mat = get_gripper_render_pose(voxel_scale,
                                                   SCENE_BOUNDS[:3],
                                                   continuous_trans,
                                                   continuous_quat)

        rendered_img_0 = visualise_voxel(vis_voxel_grid,
                                         None,
                                         [pred_trans_coord],
                                         gt_trans_coord,
                                         alpha = 0.2,
                                         voxel_size=voxel_size,
                                         rotation_amount=np.deg2rad(0),
                                         render_gripper=True,
                                         gripper_pose=gripper_pose_mat,
                                         gripper_mesh_scale=voxel_scale)

        rendered_img_270 = visualise_voxel(vis_voxel_grid,
                                           None,
                                           [pred_trans_coord],
                                           gt_trans_coord,
                                           alpha = 0.2,
                                           voxel_size=voxel_size,
                                           rotation_amount=np.deg2rad(45+180),
                                           render_gripper=True,
                                           gripper_pose=gripper_pose_mat,
                                           gripper_mesh_scale=voxel_scale)

        # Plot figures into a NumPy array
        fig = plt.figure(figsize=(20, 15))
        fig.add_subplot(1, 2, 1)
        plt.imshow(rendered_img_0)
        plt.axis('off')
        plt.title("Front view")
        fig.add_subplot(1, 2, 2)
        plt.imshow(rendered_img_270)
        plt.axis('off')
        plt.title("Side view")

        # Add timestamp as text with white font and black background
        if error:
            fig.text(0.02, 0.95, f"Timestep: {ts}, Prediction-score: {round(trans_confidence,4)}, Error: {np.round(error, 3)}", ha='left', fontsize=16, color='white', weight='bold',
                    bbox=dict(facecolor='black', edgecolor='none', boxstyle='round,pad=0.3'))
        else:
            fig.text(0.02, 0.95, f"Timestep: {ts}, Prediction-score: {round(trans_confidence,4)}", ha='left', fontsize=16, color='white', weight='bold',
                    bbox=dict(facecolor='black', edgecolor='none', boxstyle='round,pad=0.3'))

        # Convert the matplotlib figure to a NumPy array
        fig.canvas.draw()
        img_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        
        video_writer.append_data(img_array)  # Add frame to video
        # plt.show()
        plt.close(fig)  # Close the figure to free memory
        

print(f"Video saved to {video_output_path}")


### Validation: Run Inference with same batches as during training

In [7]:
import shutil

import clip

from arm.replay_buffer import create_replay, fill_replay, uniform_fill_replay, fill_replay_copy_with_crop_from_approach, fill_replay_only_approach_test

from yarr.replay_buffer.wrappers.pytorch_replay_buffer import PyTorchReplayBuffer

In [8]:
TARGET_OBJ_KEYPOINTS=False # Real - (changed later)
TARGET_OBJ_USE_LAST_KP=False # Real - (changed later)
TARGET_OBJ_IS_AVAIL = True # HandoverSim - (changed later)

STOPPING_DELTA = 0.001

In [9]:
def load_replay_buffer(settings): # Taken from analyse_data.ipynb

    # BATCH SETTINGS
    FILL_REPLAY_SETTING = settings['fill_replay_setting']
    CAMERAS = settings['cameras']
    USE_APPROACH = settings['keypoint_approach']
    DEMO_AUGMENTATION_EVERY_N = settings['demo_augm_n']

    # Summary of run properties
    print("\nExperiment Setup")
    print(f"Task: {TASK} - SETUP: {SETUP} - Cameras: {len(CAMERAS)}")
    print("Run Properties")
    print(f"Fill replay setting: {FILL_REPLAY_SETTING} - DEMO_AUGM_N: {DEMO_AUGMENTATION_EVERY_N}")

    #___REPLAY-BUFFER___
    train_replay_storage_dir = os.path.join(WORKSPACE_DIR,'replay_train')
    if os.path.exists(train_replay_storage_dir):
        print(f"Emptying {train_replay_storage_dir}")
        shutil.rmtree(train_replay_storage_dir)
    if not os.path.exists(train_replay_storage_dir):
        print(f"Could not find {train_replay_storage_dir}, creating directory.")
        os.mkdir(train_replay_storage_dir)

    test_replay_storage_dir = os.path.join(WORKSPACE_DIR,'replay_test')
    if os.path.exists(test_replay_storage_dir):
        print(f"Emptying {test_replay_storage_dir}")
        shutil.rmtree(test_replay_storage_dir)
    if not os.path.exists(test_replay_storage_dir):
        print(f"Could not find {test_replay_storage_dir}, creating directory.")
        os.mkdir(test_replay_storage_dir)

    train_replay_buffer = create_replay(batch_size=BATCH_SIZE,
                                        timesteps=1,
                                        save_dir=train_replay_storage_dir,
                                        cameras=CAMERAS,
                                        voxel_sizes=VOXEL_SIZES,
                                        image_size=IMAGE_SIZE,
                                        low_dim_size=LOW_DIM_SIZE)

    test_replay_buffer = create_replay(batch_size=BATCH_SIZE,
                                    timesteps=1,
                                    save_dir=test_replay_storage_dir,
                                    cameras=CAMERAS,
                                    voxel_sizes=VOXEL_SIZES,
                                    image_size=IMAGE_SIZE,
                                    low_dim_size=LOW_DIM_SIZE)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model, preprocess = clip.load("RN50", device=device) # CLIP-ResNet50

    print("-- Train Buffer --")
    if FILL_REPLAY_SETTING.lower() == "uniform":
        uniform_fill_replay(
            data_path=train_data_path,
            episode_folder=EPISODE_FOLDER,
            replay=train_replay_buffer,
            d_indexes=TRAIN_INDEXES,
            demo_augmentation=True,
            demo_augmentation_every_n=DEMO_AUGMENTATION_EVERY_N,
            cameras=CAMERAS,
            rlbench_scene_bounds=SCENE_BOUNDS,
            voxel_sizes=VOXEL_SIZES,
            rotation_resolution=ROTATION_RESOLUTION,
            crop_augmentation=False,
            depth_scale=DEPTH_SCALE,
            use_approach=USE_APPROACH,
            approach_distance=0.3,
            stopping_delta=STOPPING_DELTA,
            target_obj_keypoint=TARGET_OBJ_KEYPOINTS,
            target_obj_use_last_kp=TARGET_OBJ_USE_LAST_KP,
            target_obj_is_avail=TARGET_OBJ_IS_AVAIL,
            clip_model=clip_model,
            device=device,
            )
    elif FILL_REPLAY_SETTING.lower() == "crop":
        fill_replay_copy_with_crop_from_approach(
            data_path=train_data_path,
            episode_folder=EPISODE_FOLDER,
            replay=train_replay_buffer,
            d_indexes=TRAIN_INDEXES,
            demo_augmentation=True,
            demo_augmentation_every_n=DEMO_AUGMENTATION_EVERY_N,
            cameras=CAMERAS,
            rlbench_scene_bounds=SCENE_BOUNDS,
            voxel_sizes=VOXEL_SIZES,
            rotation_resolution=ROTATION_RESOLUTION,
            crop_augmentation=False,
            depth_scale=DEPTH_SCALE,
            use_approach=USE_APPROACH,
            approach_distance=0.3,
            stopping_delta=STOPPING_DELTA,
            target_obj_keypoint=TARGET_OBJ_KEYPOINTS,
            target_obj_use_last_kp=TARGET_OBJ_USE_LAST_KP,
            target_obj_is_avail=TARGET_OBJ_IS_AVAIL,
            clip_model=clip_model,
            device=device,
            )
    elif FILL_REPLAY_SETTING.lower() == "standard":
        fill_replay_only_approach_test(
        # fill_replay(
            data_path=train_data_path,
            episode_folder=EPISODE_FOLDER,
            replay=train_replay_buffer,
            d_indexes=TRAIN_INDEXES,
            demo_augmentation=True,
            demo_augmentation_every_n=DEMO_AUGMENTATION_EVERY_N,
            cameras=CAMERAS,
            rlbench_scene_bounds=SCENE_BOUNDS,
            voxel_sizes=VOXEL_SIZES,
            rotation_resolution=ROTATION_RESOLUTION,
            crop_augmentation=False,
            depth_scale=DEPTH_SCALE,
            use_approach=USE_APPROACH,
            approach_distance=0.3,
            stopping_delta=STOPPING_DELTA,
            target_obj_keypoint=TARGET_OBJ_KEYPOINTS,
            target_obj_use_last_kp=TARGET_OBJ_USE_LAST_KP,
            target_obj_is_avail=TARGET_OBJ_IS_AVAIL,
            clip_model=clip_model,
            device=device,
            )
    else:
        raise ValueError("Unkown setting for fill replay buffer")

        
    print("-- Test Buffer --")
    if FILL_REPLAY_SETTING.lower() == "uniform":
        uniform_fill_replay(
            data_path=test_data_path,
            episode_folder=EPISODE_FOLDER,
            replay=test_replay_buffer,
            d_indexes=TEST_INDEXES,
            demo_augmentation=True,
            demo_augmentation_every_n=DEMO_AUGMENTATION_EVERY_N,
            cameras=CAMERAS,
            rlbench_scene_bounds=SCENE_BOUNDS,
            voxel_sizes=VOXEL_SIZES,
            rotation_resolution=ROTATION_RESOLUTION,
            crop_augmentation=False,
            depth_scale=DEPTH_SCALE,
            use_approach=USE_APPROACH,
            approach_distance=0.3,
            stopping_delta=STOPPING_DELTA,
            target_obj_keypoint=TARGET_OBJ_KEYPOINTS,
            target_obj_use_last_kp=TARGET_OBJ_USE_LAST_KP,
            target_obj_is_avail=TARGET_OBJ_IS_AVAIL,
            clip_model=clip_model,
            device=device,
            )
    elif FILL_REPLAY_SETTING.lower() == "crop":
        fill_replay_copy_with_crop_from_approach(
            data_path=test_data_path,
            episode_folder=EPISODE_FOLDER,
            replay=test_replay_buffer,
            d_indexes=TEST_INDEXES,
            demo_augmentation=True,
            demo_augmentation_every_n=DEMO_AUGMENTATION_EVERY_N,
            cameras=CAMERAS,
            rlbench_scene_bounds=SCENE_BOUNDS,
            voxel_sizes=VOXEL_SIZES,
            rotation_resolution=ROTATION_RESOLUTION,
            crop_augmentation=False,
            depth_scale=DEPTH_SCALE,
            use_approach=USE_APPROACH,
            approach_distance=0.3,
            stopping_delta=STOPPING_DELTA,
            target_obj_keypoint=TARGET_OBJ_KEYPOINTS,
            target_obj_use_last_kp=TARGET_OBJ_USE_LAST_KP,
            target_obj_is_avail=TARGET_OBJ_IS_AVAIL,
            clip_model=clip_model,
            device=device,
            )
    elif FILL_REPLAY_SETTING.lower() == "standard":
        fill_replay_only_approach_test(
        # fill_replay(
            data_path=test_data_path,
            episode_folder=EPISODE_FOLDER,
            replay=test_replay_buffer,
            d_indexes=TEST_INDEXES,
            demo_augmentation=True,
            demo_augmentation_every_n=DEMO_AUGMENTATION_EVERY_N,
            cameras=CAMERAS,
            rlbench_scene_bounds=SCENE_BOUNDS,
            voxel_sizes=VOXEL_SIZES,
            rotation_resolution=ROTATION_RESOLUTION,
            crop_augmentation=False,
            depth_scale=DEPTH_SCALE,
            use_approach=USE_APPROACH,
            approach_distance=0.3,
            stopping_delta=STOPPING_DELTA,
            target_obj_keypoint=TARGET_OBJ_KEYPOINTS,
            target_obj_use_last_kp=TARGET_OBJ_USE_LAST_KP,
            target_obj_is_avail=TARGET_OBJ_IS_AVAIL,
            clip_model=clip_model,
            device=device,
            )
    else:
        raise ValueError("Unkown setting for fill replay buffer")


    # delete the CLIP model since we have already extracted language features
    del clip_model

    # wrap buffer with PyTorch dataset and make iterator
    train_wrapped_replay = PyTorchReplayBuffer(train_replay_buffer)
    train_dataset = train_wrapped_replay.dataset()
    train_data_iter = iter(train_dataset)

    test_wrapped_replay = PyTorchReplayBuffer(test_replay_buffer)
    test_dataset = test_wrapped_replay.dataset()
    test_data_iter = iter(test_dataset)

    return train_data_iter, test_data_iter

In [10]:
import json

run_dir = "/home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-16_11-25/" # {crop skip 10 new}

path_settings = os.path.join(os.path.dirname(run_dir), "training_settings.json")
with open(path_settings, 'r') as f:
    settings = json.load(f)

train_data_iter, test_data_iter = load_replay_buffer(settings)


Experiment Setup
Task: handing_over_banana - SETUP: s1 - Cameras: 3
Run Properties
Fill replay setting: crop - DEMO_AUGM_N: 10
Emptying /home/ywatabe/Projects/PerAct/replay_train
Could not find /home/ywatabe/Projects/PerAct/replay_train, creating directory.
Emptying /home/ywatabe/Projects/PerAct/replay_test
Could not find /home/ywatabe/Projects/PerAct/replay_test, creating directory.
-- Train Buffer --
Filling replay ...
Filling demo 45
Found 2 keypoints: [34, 58]. Using keypoint:58 and sampling till 34
Using available
Filling demo 46
Found 2 keypoints: [35, 61]. Using keypoint:61 and sampling till 35
Using available
Filling demo 47
Found 2 keypoints: [29, 60]. Using keypoint:60 and sampling till 29
Using available
Filling demo 48
Found 2 keypoints: [28, 60]. Using keypoint:60 and sampling till 28
Using available
Filling demo 49
Found 2 keypoints: [17, 54]. Using keypoint:54 and sampling till 17
Using available
Filling demo 145
Found 2 keypoints: [26, 56]. Using keypoint:56 and sampli

In [11]:
model_runs = natsorted([os.path.join(run_dir, run) for run in os.listdir(run_dir) if "run" in run])

chosen_model = "best_model_train"

model_run_distances = dict()
model_run_scores = dict()

for model_run_iter in model_runs:
    
    try:
        peract_agent.load_weights(os.path.join(model_run_iter, chosen_model))
    except:
        continue

    distances_run = []
    scores_run = []
    
    for i in range(20): # collest using 100 samples
        print(i)
        batch = next(train_data_iter) # NOTE: Choose which set to infer
        lang_goal = batch['lang_goal'][0][0][0]
        batch = {k: v.to(device) for k, v in batch.items() if type(v) == torch.Tensor}
        
        update_dict = peract_agent.update(None, batch, backprop=False)
        prediction_score = round(update_dict["q_trans"].max().item(), 4)
        pred_trans = update_dict["pred_action"]["trans"].detach().cpu().numpy()[0]
        gt_trans = update_dict["expert_action"]["action_trans"].detach().cpu().numpy()[0]
        dist = np.round(np.linalg.norm(pred_trans-gt_trans), 4)
        
        distances_run.append(dist)
        scores_run.append(prediction_score)
    
    zipped_lists = zip(distances_run, scores_run)

    sorted_lists = sorted(zipped_lists, key=lambda x: x[0])

    sorted_distances, sorted_scores = zip(*sorted_lists)
    model_run_distances[os.path.dirname(model_run_iter)] = sorted_distances
    model_run_scores[os.path.dirname(model_run_iter)] = sorted_scores


loaded weights from /home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-16_11-25/run0/best_model_train/peract_agent.pt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
loaded weights from /home/ywatabe/Projects/PerAct/outputs/models/handing_over_banana/2024-12-16_11-25/run1000/best_model_train/peract_agent.pt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

In [ ]:
from rlbench.utils import get_stored_demo
from rlbench.backend.utils import extract_obs

batch = next(train_data_iter)

# what to visualize
episode_idx_to_visualize = 846#INDEXES[0] # out of 10 demos
# ts = 70#25 # timestep out of total timesteps

# get demo
demo = get_stored_demo(data_path=test_data_path,
                    index=episode_idx_to_visualize,
                    cameras=CAMERAS,
                    depth_scale=DEPTH_SCALE,)

episode_length = list(range(len(demo._observations)))
for ts in episode_length:

    # extract obs at timestep
    obs_dict = extract_obs(demo._observations[ts], CAMERAS, t=ts)
    gripper_pose = demo[ts].gripper_pose
    gripper_open = demo[ts].gripper_open
    gripper_joint_positions = demo[ts].gripper_joint_positions

    # obs_dict["gripper_pose"] = gripper_pose
    obs_dict["gripper_open"] = gripper_open
    obs_dict["gripper_joint_positions"] = gripper_joint_positions

    # plot rgb and depth at timestep
    fig = plt.figure(figsize=(20, 10))
    rows, cols = 2, len(CAMERAS)

    plot_idx = 1
    for camera in CAMERAS:
        # rgb
        rgb_name = "%s_%s" % (camera, 'rgb')
        rgb = np.transpose(obs_dict[rgb_name], (1, 2, 0))
        fig.add_subplot(rows, cols, plot_idx)
        plt.imshow(rgb)
        plt.axis('off')
        plt.title("%s_rgb | step %s" % (camera, ts))

        # depth
        depth_name = "%s_%s" % (camera, 'depth')
        depth = np.transpose(obs_dict[depth_name], (1, 2, 0))
        fig.add_subplot(rows, cols, plot_idx+len(CAMERAS))
        plt.imshow(depth)
        plt.axis('off')
        plt.title("%s_depth | step %s" % (camera, ts))

        plot_idx += 1

    plt.show()

    print(obs_dict)

    (continuous_trans, continuous_quat, gripper_open), (voxel_grid, coord_indices, rot_and_grip_indices, gripper_open) = peract_agent_inference.forward(obs_dict, ts)

    from arm.utils import visualise_voxel
    from arm.utils import discrete_euler_to_quaternion, get_gripper_render_pose

    # things to visualize
    vis_voxel_grid = voxel_grid[0].detach().cpu().numpy()
    pred_trans_coord = coord_indices[0].detach().cpu().numpy().tolist()

    # discrete to continuous
    continuous_trans = continuous_trans[0].detach().cpu().numpy()
    continuous_quat = discrete_euler_to_quaternion(rot_and_grip_indices[0][:3].detach().cpu().numpy(),
                                                resolution=peract_agent_inference._rotation_resolution)
    gripper_open = bool(rot_and_grip_indices[0][-1].detach().cpu().numpy())
    ignore_collision = bool(test_update_dict['pred_action']['collision'][0][0].detach().cpu().numpy())

    # # gripper visualization pose
    voxel_size = 0.045
    voxel_scale = voxel_size * 100
    gripper_pose_mat = get_gripper_render_pose(voxel_scale,
                                            SCENE_BOUNDS[:3],
                                            continuous_trans,
                                            continuous_quat)

    # #@markdown #### Show Q-Prediction and Best Action
    show_expert_action = True  #@param {type:"boolean"}
    show_q_values = False  #@param {type:"boolean"}
    render_gripper = False  #@param {type:"boolean"}
    rotation_amount = -90 #@param {type:"slider", min:-180, max:180, step:5}

    rendered_img_0 = visualise_voxel(vis_voxel_grid,
                                None,
                                [pred_trans_coord],
                                None,
                                voxel_size=voxel_size,
                                rotation_amount=np.deg2rad(0),
                                render_gripper=render_gripper,
                                gripper_pose=gripper_pose_mat,
                                gripper_mesh_scale=voxel_scale,
                                perspective=False)

    rendered_img_270 = visualise_voxel(vis_voxel_grid,
                                None,
                                [pred_trans_coord],
                                None,
                                voxel_size=voxel_size,
                                rotation_amount=np.deg2rad(45),
                                render_gripper=render_gripper,
                                gripper_pose=gripper_pose_mat,
                                gripper_mesh_scale=voxel_scale)


    fig = plt.figure(figsize=(20, 15))
    fig.add_subplot(1, 2, 1)
    plt.imshow(rendered_img_0)
    plt.axis('off')
    plt.title("Front view")
    fig.add_subplot(1, 2, 2)
    plt.imshow(rendered_img_270)
    plt.axis('off')
    plt.title("Side view")


    # #@markdown #### Show Q-Prediction and Best Action
    show_expert_action = True  #@param {type:"boolean"}
    show_q_values = True  #@param {type:"boolean"}
    render_gripper = True  #@param {type:"boolean"}
    rotation_amount = -90 #@param {type:"slider", min:-180, max:180, step:5}

    rendered_img_0 = visualise_voxel(vis_voxel_grid,
                                None,
                                [pred_trans_coord],
                                None,
                                voxel_size=voxel_size,
                                rotation_amount=np.deg2rad(0),
                                render_gripper=render_gripper,
                                gripper_pose=gripper_pose_mat,
                                gripper_mesh_scale=voxel_scale)

    rendered_img_270 = visualise_voxel(vis_voxel_grid,
                                None,
                                [pred_trans_coord],
                                None,
                                voxel_size=voxel_size,
                                rotation_amount=np.deg2rad(45),
                                render_gripper=render_gripper,
                                gripper_pose=gripper_pose_mat,
                                gripper_mesh_scale=voxel_scale)


    fig = plt.figure(figsize=(20, 15))
    fig.add_subplot(1, 2, 1)
    plt.imshow(rendered_img_0)
    plt.axis('off')
    plt.title("Front view")
    fig.add_subplot(1, 2, 2)
    plt.imshow(rendered_img_270)
    plt.axis('off')
    plt.title("Side view")

    print(f"Lang goal: {lang_goal}")


In [ ]:
# from arm.utils import discrete_euler_to_quaternion, get_gripper_render_pose

# batch = next(test_data_iter)

# lang_goal = batch['lang_goal'][0][0][0]
# print(lang_goal)

# batch = {k: v.to(device) for k, v in batch.items() if type(v) == torch.Tensor}
# test_update_dict = peract_agent_inference.update(0, batch, backprop=False) # Here backprop == False: for evaluation, hence test_loss == total_loss

# # Log test metrics
# test_metrics = {
#     "total_loss": test_update_dict['total_loss'],
#     "trans_loss": test_update_dict['trans_loss'],
#     "rot_loss": test_update_dict['rot_loss'],
#     "col_loss": test_update_dict['col_loss']
# }
# for episode_kp, value in test_metrics.items():
#     print(episode_kp, value)


# from arm.utils import visualise_voxel

# # # things to visualize
# vis_voxel_grid = test_update_dict['voxel_grid'][0].detach().cpu().numpy()
# vis_trans_q = test_update_dict['q_trans'][0].detach().cpu().numpy()
# pred_trans_coord = test_update_dict['pred_action']['trans'][0].detach().cpu().numpy().tolist()
# vis_gt_coord = test_update_dict['expert_action']['action_trans'][0].detach().cpu().numpy()

# # discrete to continuous
# continuous_trans = test_update_dict['pred_action']['continuous_trans'][0].detach().cpu().numpy()
# continuous_quat = discrete_euler_to_quaternion(test_update_dict['pred_action']['rot_and_grip'][0][:3].detach().cpu().numpy(),
#                                             resolution=peract_agent_inference._rotation_resolution)
# gripper_open = bool(test_update_dict['pred_action']['rot_and_grip'][0][-1].detach().cpu().numpy())
# ignore_collision = bool(test_update_dict['pred_action']['collision'][0][0].detach().cpu().numpy())

# # gripper visualization pose
# voxel_size = 0.045
# voxel_scale = voxel_size * 100
# # print(continuous_trans, continuous_quat)
# gripper_pose = batch['gripper_state'][:, -1][0].detach().cpu().numpy()
# continuous_trans = gripper_pose[:3]
# continuous_quat = gripper_pose[3:7]
# gripper_pose_mat = get_gripper_render_pose(voxel_scale,
#                                         SCENE_BOUNDS[:3],
#                                         continuous_trans,
#                                         continuous_quat)


# # #@markdown #### Show Q-Prediction and Best Action
# show_expert_action = True  #@param {type:"boolean"}
# show_q_values = True  #@param {type:"boolean"}
# render_gripper = True  #@param {type:"boolean"}
# rotation_amount = -90 #@param {type:"slider", min:-180, max:180, step:5}

# rendered_img_0 = visualise_voxel(vis_voxel_grid,
#                             vis_trans_q if show_q_values else None,
#                             [pred_trans_coord],
#                             vis_gt_coord if show_expert_action else None,
#                             voxel_size=voxel_size,
#                             rotation_amount=np.deg2rad(0),
#                             render_gripper=render_gripper,
#                             gripper_pose=gripper_pose_mat,
#                             gripper_mesh_scale=voxel_scale)

# rendered_img_270 = visualise_voxel(vis_voxel_grid,
#                             vis_trans_q if show_q_values else None,
#                             [pred_trans_coord],
#                             vis_gt_coord if show_expert_action else None,
#                             voxel_size=voxel_size,
#                             rotation_amount=np.deg2rad(45),
#                             render_gripper=render_gripper,
#                             gripper_pose=gripper_pose_mat,
#                             gripper_mesh_scale=voxel_scale)


# fig = plt.figure(figsize=(20, 15))
# fig.add_subplot(1, 2, 1)
# plt.imshow(rendered_img_0)
# plt.axis('off')
# plt.title("Front view")
# fig.add_subplot(1, 2, 2)
# plt.imshow(rendered_img_270)
# plt.axis('off')
# plt.title("Side view")

# # print(f"Lang goal: {lang_goal}")